In [70]:
import numpy as np
import os
import pandas as pd 

import warnings
warnings.filterwarnings('ignore')

In [71]:
URL_BASE = 'https://raw.githubusercontent.com/pashkovsn/pashkov.sergei/refs/heads/main/Study/karpov.courses/lesson1/'


def read_database(file_name):
    return pd.read_csv(os.path.join(URL_BASE, file_name))

In [72]:
try:
    df_sales = read_database('2022-04-01T12_df_sales.csv')
    print(df_sales.head())
except FileNotFoundError as e:
    print(f"File not found: {e}")

try:
    df_web_logs = read_database('2022-04-01T12_df_web_logs.csv')
    print(df_web_logs.head())
except FileNotFoundError as e:
    print(f"File not found: {e}")

df_sales['date'] = pd.to_datetime(df_sales['date'])
df_web_logs['date'] = pd.to_datetime(df_web_logs['date'])

   sale_id                 date  count_pizza  count_drink  price user_id
0  1000001  2022-02-04 10:00:24            1            0    720  1c1543
1  1000002  2022-02-04 10:02:28            1            1    930  a9a6e8
2  1000003  2022-02-04 10:02:35            3            1   1980  23420a
3  1000004  2022-02-04 10:03:06            1            1    750  3e8ed5
4  1000005  2022-02-04 10:03:23            1            1    870  cbc468
  user_id page                 date  load_time
0  f25239    m  2022-02-03 23:45:37       80.8
1  06d6df    m  2022-02-03 23:49:56       70.5
2  06d6df    m  2022-02-03 23:51:16       89.7
3  f25239    m  2022-02-03 23:51:43       74.4
4  697870    m  2022-02-03 23:53:12       66.8


In [73]:
df_web_logs.head()

,user_id,page,date,load_time
0,f25239,m,2022-02-03 23:45:37,80.8
1,06d6df,m,2022-02-03 23:49:56,70.5
2,06d6df,m,2022-02-03 23:51:16,89.7
3,f25239,m,2022-02-03 23:51:43,74.4
4,697870,m,2022-02-03 23:53:12,66.8


In [74]:
df_web_logs = df_web_logs.sort_values(by=['user_id', 'date'])

In [75]:
def get_sessions(user_data):
    sessions = []
    session_start = user_data.iloc[0]['date']
    
    for i in range(1, len(user_data)):
        # Если разница во времени между текущим и предыдущим посещением больше порога, считаем сессию завершенной
        time_diff = (user_data.iloc[i]['date'] - user_data.iloc[i-1]['date']).total_seconds()
        
        # Если разрыв больше порога (например, 30 минут)
        if time_diff > 1800:  # 30 минут (1800 секунд)
            session_end = user_data.iloc[i-1]['date']
            sessions.append((user_id, session_start, session_end))
            session_start = user_data.iloc[i]['date']  # Новая сессия начинается с текущего посещения
    
    # Добавляем последнюю сессию
    session_end = user_data.iloc[-1]['date']
    sessions.append((session_start, session_end))
    
    return pd.DataFrame(sessions, columns=['user_id', 'session_start', 'session_end'])


def get_sessions(user_data):
    sessions = []
    session_start = user_data.iloc[0]['date']
    user_id = user_data.iloc[0]['user_id']  # Получаем user_id из группы

    for i in range(1, len(user_data)):
        # Если разница во времени между текущим и предыдущим посещением больше порога, считаем сессию завершенной
        time_diff = (user_data.iloc[i]['date'] - user_data.iloc[i-1]['date']).total_seconds()

        if time_diff > 1800:  # 30 минут (1800 секунд)
            session_end = user_data.iloc[i-1]['date']
            sessions.append((user_id, session_start, session_end))
            session_start = user_data.iloc[i]['date']  # Новая сессия начинается с текущего посещения

    # Добавляем последнюю сессию
    session_end = user_data.iloc[-1]['date']
    sessions.append((user_id, session_start, session_end))

    return pd.DataFrame(sessions, columns=['user_id', 'session_start', 'session_end'])

In [76]:
df_sessions = df_web_logs.groupby('user_id').apply(get_sessions).reset_index(drop=True)

In [77]:
merged_df = pd.merge(
    df_sales,
    df_sessions,
    on='user_id',
    how='inner'
)

merged_df = merged_df[
    (merged_df['date'] >= merged_df['session_start']) &
    ((merged_df['date'] - merged_df['session_start']).dt.total_seconds() <= 7200)  # 2 часа
]

merged_df = (
    merged_df[['date', 'user_id', 'session_start']]
    .groupby(['user_id','date'])
    .agg(session_start=('session_start', min))
    .reset_index()
)
merged_df.head(2)

,user_id,date,session_start
0,000096,2022-03-04 11:15:55,2022-03-04 10:58:01
1,000096,2022-03-22 13:16:09,2022-03-22 12:57:54


In [78]:
merged_df['sale_speed'] = (merged_df['date'] - merged_df['session_start']).dt.seconds
merged_df['sale_speed'].mean()/60

16.559328973854573